In [ ]:
#default_exp experiments

# Experiments

> Experiments to be run as CLI scripts (too long to run in a Jupyter session)

In [ ]:
#export
import papermill as pm
import numpy as np
from fastcore.all import *

In [ ]:
import wandb
import numpy as np

### Analyse the impact of the number of points on the performance of the motion classifier

The results will be logged in a run group in wandb, and from there the visualizations will be made

In [ ]:
#export experiments
@call_parse
def x_sequence_length(min_rp:Param("Minimum resampling period", int)=1,
                      max_rp:Param("Maximum resampling period", int)=10,
                      rep_times:Param("Number of times ru run per rp", int)=10,
                      nbs_folder:Param("Notebook folder path", str)='.'):
    for x in range(min_rp, max_rp+1):
        for _ in range(rep_times):
            pm.execute_notebook(input_path=f'{Path(nbs_folder)}/_motion_classification_time_series.ipynb',
                                output_path='--',
                                parameters=dict(
                                    use_wandb=True,
                                    wandb_group='x_sequence_length',
                                    resampling_period=x
                                ))

In [ ]:
%%capture
x_sequence_length(max_rp=2, rep_times=1)

### Analyse how the inclusion of more and more data helps to predict better the motions of the next epsilon (x_next_epsilon)

In [ ]:
#export experiments
@call_parse
def x_next_epsilon(nbs_folder:Param("Notebook folder path", str)='.'):
    base_path = Path.home()
    ds_path = base_path/'data/fli_rotnum/model_1'
    fnames = L(
        (ds_path/'eps=0.021/Poincare.plt', ds_path/'eps=0.021/index_fli_rotnum_birav.plt'),
        (ds_path/'eps=0.022/Poincare.plt', ds_path/'eps=0.022/index_fli_rotnum_birav.plt'),
        (ds_path/'eps=0.023/Poincare.plt', ds_path/'eps=0.023/index_fli_rotnum_birav.plt'),
        (ds_path/'eps=0.024/Poincare.plt', ds_path/'eps=0.024/index_fli_rotnum_birav.plt'),
        (ds_path/'eps=0.025/Poincare.plt', ds_path/'eps=0.025/index_fli_rotnum_birav.plt'),
    )
    for i in range(2, len(fnames)+1):
        x_input = L(fnames[:i])
        val_dataset = x_input[-1][0].parent.name
        print(val_dataset)
        pm.execute_notebook(input_path=f'{Path(nbs_folder)}/_motion_classification_time_series.ipynb',
                            output_path='./__.ipynb',
                            parameters=dict(
                                use_wandb=True,
                                wandb_group='x_next_epsilon',
                                val_dataset = val_dataset,
                                fnames_poinc_map=list(x_input.itemgot(0).map(str)),
                                fnames_index=list(x_input.itemgot(1).map(str))
                            ))

In [ ]:
x_next_epsilon()

## Try different validation datasets

In [ ]:
x_val_datasets = {
    'name': 'x_val_datasets',
    'description': 'Test different validation datasets,  \
                    keeping the training set constant',
    'method': 'grid',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'val_ds': {
            'distribution': 'categorical',
            'values': ['eps=0.021', 'eps=0.022', 'eps=0.023', 'eps=0.024', 'eps=0.025']
        }
    }
}

## x_next_epsilon

In [ ]:
x_next_epsilon = {
    'name': 'x_next_epsilon',
    'description': 'keeping the training set constant to eps=0.021, validate on incremental \
    values of epsilon from eps=0.021 (training=validaton) to eps=0.025, to see how the distance \
    to the training parameters affect performance.',
    'method': 'grid',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'val_ds': {
            'distribution': 'categorical',
            'values': ['eps=0.021', 'eps=0.022', 'eps=0.023', 'eps=0.024', 'eps=0.025']
        }
    }
}

In [ ]:
sweep_id=wandb.sweep(x_next_epsilon, project='mlchaos')

In [ ]:
wandb.agent(sweep_id,
            function=partial(pm.execute_notebook, 
                             input_path='./_motion_classification_time_series.ipynb', 
                             output_path='__.ipynb', 
                             parameters=dict(
                                 fnames_poinc_map=['/home/user/data/fli_rotnum/model_1/eps=0.021/Poincare.plt'],
                                 fnames_index=['/home/user/data/fli_rotnum/model_1/eps=0.021/index_fli_rotnum_birav.plt'], 
                                 resampling_period=2, 
                                 use_wandb=True)
                            ))

## Analyse the effect of the labelling method

In [ ]:
x_labelling_method = {
    'name': 'x_labelling_method',
    'description': 'Try different labelling methods, possibly over different validation sets',
    'method': 'grid',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'labelling_method': {
            'distribution': 'categorical',
            'values': ['fli', 'fli_no_uncertainty', 'freq_ana_eq_3.5', 'freq_ana_eq_3.6', 'birav']
        },
        'val_ds': {
            'distribution': 'categorical',
            'values': list(L(np.arange(0.021, 0.026, 0.001).round(3), use_list=True).map('eps={}'))
        }
    }
}

In [ ]:
# list(L(np.arange(0.021, 0.026, 0.001).round(3), use_list=True).map('eps={}'))

In [ ]:
sweep_id=wandb.sweep(x_labelling_method)

In [ ]:
wandb.agent('1qg2e9i4',
            project='mlchaos',
            function=partial(pm.execute_notebook, 
                             input_path='./_motion_classification_time_series.ipynb', 
                             output_path='__.ipynb', 
                             parameters=dict(
                                 wandb_group='x_labelling_method',
                                 resampling_period=4, 
                                 use_wandb=True)
                            ))

wandb: Agent Starting Run: 0hrk0h7w with config:
wandb: 	labelling_method: freq_ana_eq_3.5
wandb: 	val_ds: eps=0.023


wandb: Agent Starting Run: ieckvslg with config:
wandb: 	labelling_method: freq_ana_eq_3.5
wandb: 	val_ds: eps=0.024


wandb: Agent Starting Run: 4e3m1d92 with config:
wandb: 	labelling_method: freq_ana_eq_3.5
wandb: 	val_ds: eps=0.025


wandb: Agent Starting Run: 96e8ad4o with config:
wandb: 	labelling_method: freq_ana_eq_3.6
wandb: 	val_ds: eps=0.021


wandb: Agent Starting Run: kjjhp79j with config:
wandb: 	labelling_method: freq_ana_eq_3.6
wandb: 	val_ds: eps=0.022


...

wandb: Agent Starting Run: 4i8f6w98 with config:
wandb: 	labelling_method: birav
wandb: 	val_ds: eps=0.022


wandb: Agent Starting Run: au13qt77 with config:
wandb: 	labelling_method: birav
wandb: 	val_ds: eps=0.023


wandb: Agent Starting Run: fkptwpd1 with config:
wandb: 	labelling_method: birav
wandb: 	val_ds: eps=0.024


wandb: Agent Starting Run: faj7ujgu with config:
wandb: 	labelling_method: birav
wandb: 	val_ds: eps=0.025


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
